In [191]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.util import ngrams
import re

### Load dataframe:

In [192]:
with open('../data_per_simpCode_00_clean_status.txt', 'r') as f:
    lines = f.read().splitlines()
df = pd.DataFrame(lines, dtype='string')
#df=pd.read_table('../data_per_simpCode_00_clean_status.txt', sep='\n', engine='python', header=None, encoding='utf8')

### Clean with regular expressions:

In [193]:
def replace_numbers(sentence):
    return re.sub('\d+', 'NUM', sentence)

def replace_float_numbers(sentence):
    return re.sub('NUM.NUM', 'NUM', sentence)

def remove_multiple_spaces(sentence):
    return re.sub('\s+', ' ', sentence)

def remove_multiple_tabs(sentence):
    return re.sub('\t+', ' ', sentence)

def space_out_semicolon(sentence):
    return re.sub(';', ' ; ', sentence)

def add_space_before_numbers(sentence):
    return re.sub(r"(\w+)NUM", r"\1 NUM", sentence)

def add_space_after_numbers(sentence):
    return re.sub(r"NUM(\w+)", r"NUM \1", sentence)

def remove_semicolons(sentence):
    return re.sub(';', ' ', sentence)

def remove_colons(sentence):
    return re.sub(':', ' ', sentence)

def remove_commas(sentence):
    return re.sub('\,', ' ', sentence)

def remove_slashes(sentence):
    return re.sub('\/+', ' ', sentence)

def remove_backslashes(sentence):
    return re.sub('\\\\+', ' ', sentence)

def remove_plus_signs(sentence):
    return re.sub('\+', ' ', sentence)

def remove_underscores(sentence):
    return re.sub('\_', ' ', sentence)

def replace_invalid_dash(sentence):
    return re.sub('–', '-', sentence)

def replace_multiple_dashes(sentence):
    return re.sub('-{2,}', ' - ', sentence)

def replace_multiple_dots(sentence):
    return re.sub('\.{2,}', '.', sentence)

def remove_dashes_followed_by_space(sentence):
    return re.sub(r"(.)-\s", r"\1 ", sentence)

def remove_dashes_following_a_space(sentence):
    return re.sub(r"\s-(.)", r" \1", sentence)

def remove_negative_examination(sentence):
    return re.sub('\(-\)', ' ', sentence)

def remove_starting_special_characters(sentence):
    return re.sub('^[\.\?\-,]+', '', sentence)

def remove_ending_special_characters(sentence):
    return re.sub('[\.\?\-,]+$', '', sentence)

def remove_dots(sentence):
    return re.sub('[\.]+', ' ', sentence)

def remove_quotation_marks(sentence):
    return re.sub('[\`\'\"]+', ' ', sentence)

def remove_dot_before_word(sentence):
    return re.sub(r"([^\wа-яА-Я])\.([\wа-яА-Я])", r"\1 \2", sentence)

def remove_dot_followed_by_dash(sentence):
    return re.sub(r"\.\-", '. ', sentence)

def space_out_dot_before_number(sentence):
    return re.sub(r"[\.][\-=]NUM", '. NUM', sentence)

def split_words_with_capital_after_lowercase(sentence):
    return re.sub(r"([а-яa-z])([А-ЯA-Z])", r"\1 \2", sentence)


df['clean_sentence'] = df[0].apply(remove_semicolons)
df['clean_sentence'] = df['clean_sentence'].apply(remove_colons)
df['clean_sentence'] = df['clean_sentence'].apply(remove_commas)
df['clean_sentence'] = df['clean_sentence'].apply(remove_slashes)
df['clean_sentence'] = df['clean_sentence'].apply(remove_backslashes)
df['clean_sentence'] = df['clean_sentence'].apply(remove_plus_signs)
df['clean_sentence'] = df['clean_sentence'].apply(remove_underscores)
df['clean_sentence'] = df['clean_sentence'].apply(replace_invalid_dash)
df['clean_sentence'] = df['clean_sentence'].apply(replace_multiple_dashes)
df['clean_sentence'] = df['clean_sentence'].apply(replace_multiple_dots)
df['clean_sentence'] = df['clean_sentence'].apply(remove_dashes_followed_by_space)
df['clean_sentence'] = df['clean_sentence'].apply(remove_dashes_following_a_space)
df['clean_sentence'] = df['clean_sentence'].apply(remove_negative_examination)
df['clean_sentence'] = df['clean_sentence'].apply(remove_starting_special_characters)
df['clean_sentence'] = df['clean_sentence'].apply(remove_ending_special_characters)
df['clean_sentence'] = df['clean_sentence'].apply(replace_numbers)
df['clean_sentence'] = df['clean_sentence'].apply(replace_float_numbers)
df['clean_sentence'] = df['clean_sentence'].apply(add_space_before_numbers)
df['clean_sentence'] = df['clean_sentence'].apply(add_space_after_numbers)
df['clean_sentence'] = df['clean_sentence'].apply(space_out_dot_before_number)
df['clean_sentence'] = df['clean_sentence'].apply(remove_dot_before_word)
df['clean_sentence'] = df['clean_sentence'].apply(remove_dot_followed_by_dash)
df['clean_sentence'] = df['clean_sentence'].apply(split_words_with_capital_after_lowercase)
# df['clean_sentence'] = df['clean_sentence'].apply(remove_dots) # commetnt this line
# df['clean_sentence'] = df['clean_sentence'].apply(space_out_semicolon)
df['clean_sentence'] = df['clean_sentence'].apply(remove_quotation_marks)
df['clean_sentence'] = df['clean_sentence'].apply(remove_multiple_tabs)
df['clean_sentence'] = df['clean_sentence'].apply(remove_multiple_spaces)

### Special cases cleaning:

In [194]:
def remove_dash_after_special_words(sentence):
    special_words = [
        'апарат', 'вени', 'възли', 'глава', 'гърло', 'далак', 'дейност', 'дроб', 'език',
        'жлеза', 'жлези', 'зрение', 'кожа', 'корем', 'крайници', 'нос', 'ода', 'одс', 
        'простата', 'пулмо', 'пулс', 'рефлекси', 'с-ма', 'слезка', 'сливици', 
        'слух', 'статус', 'сърце', 'състояние',
        'тегло', 'температура', 'тонзили', 'шия', 'чмн', 'rr']
    for word in special_words:
        sentence = re.sub(f"({word})-", r"\1 ", sentence, flags=re.IGNORECASE)
    return sentence

def replace_special_expressions(sentence):
    special_expressions = [
        ['V O S', 'VOS'],
        ['V O D', 'VOD'],
        [r"([^\wа-яА-Я])(?:рр|РР|Рр|rr|PP)", r"\1 RR"],
        [r"(?:рр|РР|Рр|rr|PP)([^\wа-яА-Я])", r"RR \1"],
        [r"([^\wа-яА-Я]*)(?:кор|Кор|КОР|kor|Кор)([^\wа-яА-Я])", r"\1 Cor "],
        [r"(?:б\.о\.|\sБО\s|\sБо\s|\sбо\s|\sб\sо\s|\sБ\sО\s)", ' б.о. '],
        [r"(?:РСД|р\.с\.д\.|рсд|PCД)", ' РСД '],
        ['ССС', ' ССС '],
    ]
    for find, replace in special_expressions:
        sentence = re.sub(find, replace, sentence)
    return sentence

df['clean_sentence'] = df['clean_sentence'].apply(remove_dash_after_special_words)
df['clean_sentence'] = df['clean_sentence'].apply(replace_special_expressions)

df['clean_sentence'] = df['clean_sentence'].apply(remove_dashes_followed_by_space)
df['clean_sentence'] = df['clean_sentence'].apply(remove_dashes_following_a_space)
df['clean_sentence'] = df['clean_sentence'].apply(remove_dot_before_word)
df['clean_sentence'] = df['clean_sentence'].apply(remove_multiple_spaces)

### Tokenize:

In [195]:
# save cleaned data so we know what tokenizer works on
df['clean_sentence'].to_csv('sentence_cleaned_status.csv')

discarded_tokens = ['.', 'NUM']

unigrams = {}
bigrams = {}
for sentence in df['clean_sentence']:
    tokens = word_tokenize(sentence)
    tokens = list(filter(lambda token: token not in discarded_tokens, tokens))

    #get unigrams
    for token in tokens:
        if token not in unigrams:
            unigrams[token] = 0
        unigrams[token] += 1

    #get bigrams
    bigram_list = list(ngrams(tokens, 2))
    for bigram in bigram_list:
        if bigram not in bigrams:
            bigrams[bigram] = 0
        bigrams[bigram] += 1

unigrams = sorted(unigrams.items(), key=lambda x: x[1], reverse=True)
bigrams = sorted(bigrams.items(), key=lambda x: x[1], reverse=True)
print('====Unigrams length====')
print(len(unigrams))
print('====First 50 unigrams====')
print(unigrams[0:50])
print('====First 50 bigrams====')
print(bigrams[0:50])

====Unigrams length====
38948
====First 50 unigrams====
[('и', 29810), ('без', 27007), ('на', 23444), ('RR', 21816), ('б.о', 21280), ('Корем', 17228), ('Cor', 16859), ('в', 14394), ('дишане', 14238), ('мек', 12802), ('РСД', 12269), ('тонове', 11823), ('с', 10757), ('отоци', 10030), ('не', 9508), ('дейност', 9071), ('се', 8551), ('ясни', 8517), ('Пулмо', 8508), ('мин', 8200), ('Крайници', 7744), ('везикуларно', 7612), ('сърдечна', 6903), ('хрипове', 6524), ('неболезнен', 6524), ('за', 6213), ('дроб', 5956), ('ритмична', 5503), ('статус', 5341), ('общо', 5233), ('двустранно', 5136), ('ССС', 5118), ('по', 4644), ('уд', 4573), ('см', 4539), ('състояние', 4447), ('палпират', 4398), ('слезка', 4385), ('вез', 4338), ('крайници', 4212), ('АН', 4083), ('чисто', 4004), ('Кожа', 3829), ('шумове', 3581), ('болезнен', 3567), ('сърд', 3539), ('отр', 3376), ('тон', 3229), ('пулсации', 3198), ('лигавици', 3120)]
====First 50 bigrams====
[(('Корем', 'мек'), 9180), (('без', 'отоци'), 7393), (('везикулар

### Collocations extraction:

In [196]:
def get_collocations(bigrams, freq_threshold=1):
    collocations = []

    bigram_map_first_word = {}
    bigram_map_second_word = {}
    for bigram, _ in bigrams:
        if bigram[0] not in bigram_map_first_word:
            bigram_map_first_word[bigram[0]] = 0
        bigram_map_first_word[bigram[0]] += 1
        if bigram[1] not in bigram_map_second_word:
            bigram_map_second_word[bigram[1]] = 0
        bigram_map_second_word[bigram[1]] += 1

    for bigram, freq in bigrams:
        if freq < freq_threshold:
            continue
        if bigram_map_first_word[bigram[0]] == 1 or bigram_map_second_word[bigram[1]] == 1:
            collocations.append(bigram)

    return collocations

collocations = get_collocations(bigrams, 5)

print('====Collocations length====')
print(len(collocations))
print('====First 50 collocations====')
print(collocations[0:50])

====Collocations length====
2549
====First 50 collocations====
[('ПИКОЧО-ПОЛОВО', 'С-МА'), ('МАНУАЛНО', 'ИЗСЛ'), ('Артериално', 'налягане'), ('(', 'Score'), ('Score', ')'), ('Съдечен', 'статус'), ('Корем', 'меко-еластичен'), ('везиколарно', 'дишане'), ('сърд', 'гран'), ('на', 'дебелото'), ('се', 'установява'), ('Ч.ДР', 'И'), ('на', 'артериите'), ('на', 'гръд.кош'), ('смущ', 'в'), ('без', 'доб'), ('доб', 'хрипове'), ('Авто', 'и'), ('рефлекси', 'Отпадна'), ('РСД', 'яс'), ('яс', 'тон'), ('сърд', 'гран.в'), ('гран.в', 'норма'), ('НА', 'ВЪРХА'), ('Цветно', 'зрение'), ('на', 'Аорта'), ('Аорта', 'RR'), ('Височина', 'см'), ('С-Фр.NUM', 'у'), ('Об.талия', 'см'), ('мм', 'Нв'), ('Нв', 'Корем'), ('мек', 'респиратнорно'), ('респиратнорно', 'подвижен'), ('за', 'ХСМ.Мек'), ('ХСМ.Мек', 'Физ'), ('без', 'хроп'), ('хроп', 'находка'), ('ритмична', 'сър.д-ст'), ('Оглед', 'палпация'), ('Пулсова', 'честота'), ('от', 'Крайничци'), ('Крайничци', 'леки'), ('Мекоеластична', 'коремна'), ('лимни', 'възли'), ('ЗА',

### Most common mistakes

In [197]:
from nltk.metrics.distance import jaccard_distance

top_unigrams_count = round(len(unigrams) * 0.03)
top_unigrams = unigrams[0:top_unigrams_count]

top_mistakes = []

for unigram, _ in top_unigrams:
    if len(unigram) < 3: continue
    mistakes = { unigram: [] }
    for other_unigram, _ in unigrams:
        distance = jaccard_distance(set(unigram), set(other_unigram))
        if distance < 0.15 and unigram != other_unigram:
            mistakes[unigram].append(other_unigram)
    top_mistakes.append(mistakes)

print('====First 50 unigram mistakes====')
print(top_mistakes[0:50])

====First 50 unigram mistakes====
[{'без': ['безз', 'безе', 'ебез']}, {'б.о': ['б.о.', 'бо.о', 'бо.', 'б.оо', 'бб.о']}, {'Корем': ['Корме']}, {'Cor': []}, {'дишане': ['вдишване', 'дишанес', 'дишанел', 'издишане', 'дишаен', 'диишане', 'диашне', 'дишване', 'дишишане', 'дишанне', 'диша-не', 'деишане', 'дишанеи', 'дишавне', 'дишание', 'иьдишане', 'дишане.', 'дишанеRR', 'дишанев', 'дишанееедри', 'дишанеез', 'дзишане', 'идишване', 'дишарне', 'дишанене', 'дишаяне', 'издишнане']}, {'мек': []}, {'РСД': ['РДС', 'СРД', 'ДРС']}, {'тонове': ['тоновете', 'тоонве', 'тононве', 'товнове', 'тоонове', 'тонве', 'ттонове', 'тнове', 'тоновет', 'тоннове', 'тоновве', 'тонвое', 'тонтове', 'тооннове']}, {'отоци': ['оттоци', 'отци', 'тоци', 'отоиц', 'оитоци', 'отооци', 'ототци']}, {'дейност': ['дейнос', 'с.дейност', 'деност', 'дейост', 'дейност.', 'дейнот', 'дейности', 'дейносст', 'дейноскт', 'десното', 'деййност', 'дейнст', 'дейножст', 'деайност', 'дейниост', 'дейностс', 'десност', 'дейнсот', 'дейностq', 'дейно